In [1]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [1]:
import pandas as pd

In [3]:
! pip install h5py

In [4]:
import pandas as pd
import h5py
import numpy as np
from PIL import Image

In [6]:
filename = './image_toy/20200109pp-set37__bin4_whole_seg.hdf'
hdf = h5py.File(filename,'r')
data = hdf['MDF']['images']['0']['image']
test = np.array(data)

In [8]:
formatted = (test * 255 / np.max(test)).astype('uint8')
img = Image.fromarray(formatted)
#img

In [9]:
import os, sys

In [12]:
file_dir = './image_toy'
suffix = '_whole.hdf'
file_list = os.listdir(file_dir)
file_list 

['20200109pp-set37__bin4_whole.hdf', '20200109pp-set37__bin4_whole_seg.hdf']

In [13]:
whole_images = [f for f in file_list if f.endswith(suffix)]

In [15]:
f = whole_images[0]

In [17]:
prefix = f.split(suffix)[0]

In [18]:
df = pd.read_csv('image_pairs.csv')
df.head()

,image,truth
0,20200109pp-set37__bin4_whole.hdf,20200109pp-set37__bin4_whole_seg.hdf


In [20]:
df.iloc[0,]['image']

'20200109pp-set37__bin4_whole.hdf'

In [21]:
file_dir = './image_toy'

suffix = '_whole.hdf'
file_list = os.listdir(file_dir)
truth_suffix = '_whole_seg.hdf'

whole_images = [f for f in file_list if f.endswith(suffix)]
record = []
for f in whole_images:
    prefix = f.split(suffix)[0]
    truth = prefix+truth_suffix
    record.append([f,truth])

data = pd.DataFrame(record)
data.columns = ['image','truth']
data.to_csv('image_pairs.csv',index=False)

In [22]:
csv_file = 'image_pairs.csv'
df = pd.read_csv(csv_file)
df

,image,truth
0,20200109pp-set37__bin4_whole.hdf,20200109pp-set37__bin4_whole_seg.hdf
